In [43]:
import itertools
import random

In [180]:
class CardGame:
    def __init__(self, num_players):
        self.num_players = num_players
        self.deck = self.create_deck()
        self.players_hands = [[] for _ in range(num_players)]
        self.face_down_cards = [[] for _ in range(num_players)]
        self.face_up_cards = [[] for _ in range(num_players)]
        self.draw_pile = []
        self.play_pile = []
        self.game_over = False
        self.play_direction = 1 # 1 for clockwise, -1 for counterclockwise
        self.setup_game()
        
    def create_deck(self):
        # Define card values and suits
        suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
        values = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'Jack', 'Queen', 'King', 'Ace']
        # Include Jokers
        deck = [(value, suit) for value in values for suit in suits] + [('Joker', 'Black'), ('Joker', 'Red')]
        # Use 2 decks for 5-8 players
        if self.num_players >= 5:
            deck *= 2
        
        return deck
    
    def setup_game(self):
        # Shuffle the deck
        random.shuffle(self.deck) 

        # Deal 3 face-down and 6 to choose face-up cards to each player
        for i in range(self.num_players):
            # Deal 3 face-down cards
            self.face_down_cards[i] = [self.deck.pop() for _ in range(3)]

            # Deal 6 cards for choosing 3 to be face-up
            face_up_candidates = [self.deck.pop() for _ in range(6)]

            # Players choose 3 of these to be face-up, for simplicity, randomly select here
            self.face_up_cards[i] = random.sample(face_up_candidates, 3)

            # The remaining 3 cards from the 6 initially dealt for choosing go into the player's hand
            self.players_hands[i] = [card for card in face_up_candidates if card not in self.face_up_cards[i]]

        # The rest of the cards form the draw pile
        self.draw_pile = self.deck
    
    def find_starting_player(self):
        # Translate card values to numbers for easier comparison
        card_value_order = {str(i): i for i in range(2, 11)}
        card_value_order.update({'Jack': 11, 'Queen': 12, 'King': 13, 'Ace': 14, 'Joker': 15})
        
        
        # Initialize counts for each card value for each player
        player_card_counts = [{value: 0 for value in card_value_order.keys()} for _ in range(self.num_players)]
        
        
        # Count the number of each card value in each player's hand
        for player_index, hand in enumerate(self.players_hands):
            for card in hand:
                if card[0] in card_value_order:  # Ignore suits
                    player_card_counts[player_index][card[0]] += 1
        
        #print(player_card_counts)
        
        # Determine who starts based on the cards
        for value in ['3', '4', '5', '6', '7', '8', '9', '10', 'Jack', 'Queen', 'King', 'Ace', 'Joker']:
            # Find players with the current card value
            players_with_card = [(index, count[value]) for index, count in enumerate(player_card_counts) if count[value] > 0]
            
            if not players_with_card:  # If no player has the card, continue to the next value
                continue
            
            # If only one player has the card, they start
            if len(players_with_card) == 1:
                return players_with_card[0][0]
            
            # If multiple players have the same card, compare counts
            max_count = max(players_with_card, key=lambda x: x[1])[1]
            players_with_max = [player for player, count in players_with_card if count == max_count]
            
            if len(players_with_max) == 1:
                return players_with_max[0]
            # If still tied, continue to the next card value
            
        # If somehow no one can start, return a random player as fallback
        return random.randint(0, self.num_players - 1)

    def start_game(self):
        
        starting_player_index = self.find_starting_player()
        current_player_index = starting_player_index
        
        while not self.game_over:
            # Player takes their turn.
            self.play_turn(current_player_index)
                    
            self.game_over = self.check_game_over()
            
            # Move to the next player
            current_player_index = (current_player_index + self.play_direction) % self.num_players
            
        print("Game Over")
        
        
    def check_game_over(self):
        # Implement logic to determine if the game has ended.
        # This could involve checking if any player has successfully played all their cards (hand, face-up, and face-down).
        for player_index in range(self.num_players):
            if (not self.players_hands[player_index] and 
                not self.face_up_cards[player_index] and 
                not self.face_down_cards[player_index]):
                print(f"Player {player_index + 1} wins and is declared the Canary!")
                return True  # A winner is found
        return False  # The game continues
    
    def get_card_value(self, card):
        """Convert card to its numerical value for comparison, considering special cards."""
        card_value_order = {'2': 15, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10,
                            'Jack': 11, 'Queen': 12, 'King': 13, 'Ace': 14, 'Joker': 16}
        
        return card_value_order.get(card[0], 0)

    def play_turn(self, current_player_index):
        print(f"Player {current_player_index + 1}'s turn:")
        
        print("Player Hand: ",self.players_hands[current_player_index])
        
        # If the player has no cards in their hand, try playing face-up cards
        if not self.players_hands[current_player_index]:
            if self.face_up_cards[current_player_index]:
                playable_cards = [card for card in self.face_up_cards[current_player_index] if self.can_play_card(card)]
                if not playable_cards:
                    print("No playable cards. Must pick up the pile.")
                    self.pick_up_pile(current_player_index)
                    return
                
                chosen_card = random.choice(playable_cards)
                
                # Check for Jokers and reverse the order of play if necessary
                if chosen_card[0] == 'Joker':
                    self.reverse_play_order()
                
                self.play_face_up_card(current_player_index, chosen_card)
                
            elif self.face_down_cards[current_player_index]:
                chosen_card = random.choice(self.face_down_cards[current_player_index])
                if self.can_play_card(chosen_card):
                    # Check for Jokers and reverse the order of play if necessary
                    if chosen_card[0] == 'Joker':
                        self.reverse_play_order()
                    self.play_face_down_card(current_player_index, chosen_card)
                else:
                    print("No playable cards. Must pick up the pile.")
                    self.players_hands[player_index].append(card_drawn)
                    self.pick_up_pile(current_player_index)
                    return
                
                
                
            else:
                print("No more cards to play.")
                self.check_game_over()
            return  # Exit the turn once a face-up or face-down card play attempt is made
        
        
        playable_cards = [card for card in self.players_hands[current_player_index] if self.can_play_card(card)]
        
        if not playable_cards:
            print("No playable cards. Must pick up the pile.")
            self.pick_up_pile(current_player_index)
            return
        
        # For simplicity, randomly choose a card to play from playable cards
        chosen_card = random.choice(playable_cards)
        # Check for Jokers and reverse the order of play if necessary
        if chosen_card[0] == 'Joker':
            self.reverse_play_order()
        self.play_card(current_player_index, chosen_card)
        print(f"Played {chosen_card}.")
        print(self.play_pile[-1])
        
        # Ensure the player has at least 3 cards in their hand if the draw pile isn't empty
        while len(self.players_hands[current_player_index]) < 3 and self.draw_pile:
            self.draw_card(current_player_index)
            print(f"Player {current_player_index + 1} draws a card. Hand now: {self.players_hands[current_player_index]}")

    def reverse_play_order(self):
        """Reverses the current order of play."""
        # This is a simplified representation. You might store the current direction of play
        # (e.g., as a class attribute) and reverse it here. For example:
        self.play_direction *= -1
    
    def draw_card(self, player_index):
        """Player draws a card from the draw pile."""
        if self.draw_pile:
            card_drawn = self.draw_pile.pop()
            self.players_hands[player_index].append(card_drawn)
        else:
            print("The draw pile is empty. No card drawn.")
    
    def can_play_card(self, card):
        """Check if the card can be played on top of the play pile."""
        if not self.play_pile:
            return True  # Any card can be played if the play pile is empty
        
        top_card = self.play_pile[-1]
        
        # Allow any card to be played if the top card is a Joker
        if top_card[0] == 'Joker':
            return True
        
        # Jokers can be played on anything
        if card[0] == 'Joker':
            return True
    
        print(self.get_card_value(card), " >= ", self.get_card_value(top_card),"?")
        return self.get_card_value(card) >= self.get_card_value(top_card)

    def play_card(self, player_index, card):
        """Play a card from the player's hand to the play pile."""
        self.players_hands[player_index].remove(card)
        self.play_pile.append(card)
    
    def pick_up_pile(self, player_index):
        """Player picks up the play pile."""
        self.players_hands[player_index].extend(self.play_pile)
        self.play_pile.clear()
        print("Picked up the pile.")
        
    def play_face_up_card(self, player_index, card):
        # Implement logic to play a face-up card
        self.face_up_cards[player_index].remove(card)
        self.play_pile.append(card)
        print(f"Player {player_index + 1} plays a face-up card.")

    def play_face_down_card(self, player_index, card):
        # Implement logic to randomly select and attempt to play a face-down card
        self.face_down_cards[player_index].remove(card)
        self.play_pile.append(card)
        print(f"Player {player_index + 1} plays a face-down card.")

game = CardGame(4)  

In [182]:
game = CardGame(4)
game.start_game()

Player 2's turn:
Player Hand:  [('3', 'Hearts'), ('5', 'Diamonds'), ('6', 'Hearts')]
Played ('3', 'Hearts').
('3', 'Hearts')
Player 2 draws a card. Hand now: [('5', 'Diamonds'), ('6', 'Hearts'), ('Joker', 'Black')]
Player 3's turn:
Player Hand:  [('8', 'Hearts'), ('8', 'Clubs'), ('Jack', 'Diamonds')]
8  >=  3 ?
8  >=  3 ?
11  >=  3 ?
Played ('Jack', 'Diamonds').
('Jack', 'Diamonds')
Player 3 draws a card. Hand now: [('8', 'Hearts'), ('8', 'Clubs'), ('King', 'Spades')]
Player 4's turn:
Player Hand:  [('Ace', 'Hearts'), ('Queen', 'Clubs'), ('6', 'Clubs')]
14  >=  11 ?
12  >=  11 ?
6  >=  11 ?
Played ('Queen', 'Clubs').
('Queen', 'Clubs')
Player 4 draws a card. Hand now: [('Ace', 'Hearts'), ('6', 'Clubs'), ('7', 'Diamonds')]
Player 1's turn:
Player Hand:  [('8', 'Diamonds'), ('7', 'Spades'), ('8', 'Spades')]
8  >=  12 ?
7  >=  12 ?
8  >=  12 ?
No playable cards. Must pick up the pile.
Picked up the pile.
Player 2's turn:
Player Hand:  [('5', 'Diamonds'), ('6', 'Hearts'), ('Joker', 'Black'

In [183]:
game.deck

[]

In [184]:
game.draw_pile

[]

In [185]:
game.face_down_cards

[[('5', 'Clubs'), ('King', 'Hearts'), ('Joker', 'Red')],
 [('10', 'Diamonds'), ('5', 'Hearts'), ('2', 'Spades')],
 [('Ace', 'Clubs'), ('9', 'Clubs'), ('2', 'Hearts')],
 []]

In [186]:
game.face_up_cards

[[('4', 'Spades'), ('7', 'Clubs'), ('Jack', 'Spades')],
 [('Jack', 'Clubs'), ('10', 'Spades'), ('Ace', 'Diamonds')],
 [('3', 'Clubs'), ('Queen', 'Spades'), ('5', 'Spades')],
 []]

In [187]:
game.players_hands

[[('3', 'Hearts'),
  ('4', 'Hearts'),
  ('7', 'Spades'),
  ('2', 'Clubs'),
  ('2', 'Diamonds'),
  ('Joker', 'Black'),
  ('3', 'Diamonds'),
  ('5', 'Diamonds'),
  ('6', 'Diamonds'),
  ('Jack', 'Diamonds'),
  ('Queen', 'Clubs')],
 [('8', 'Spades'),
  ('8', 'Diamonds'),
  ('6', 'Clubs'),
  ('9', 'Diamonds'),
  ('9', 'Spades'),
  ('7', 'Diamonds'),
  ('8', 'Clubs'),
  ('Queen', 'Diamonds'),
  ('9', 'Hearts')],
 [('10', 'Hearts'),
  ('6', 'Hearts'),
  ('Jack', 'Hearts'),
  ('3', 'Spades'),
  ('Queen', 'Hearts'),
  ('King', 'Spades'),
  ('7', 'Hearts'),
  ('4', 'Clubs'),
  ('10', 'Clubs'),
  ('King', 'Clubs'),
  ('Ace', 'Hearts')],
 []]

In [188]:
game.play_pile

[('4', 'Diamonds'),
 ('6', 'Spades'),
 ('8', 'Hearts'),
 ('King', 'Diamonds'),
 ('Ace', 'Spades')]